# Labs: Visualize Correlation

In this notebook, you will visualize the correlation between two factors in Quantopian. Specifically, we want to see how today's news sentiment (from the [SentDex](https://www.quantopian.com/docs/data-reference/sentdex) dataset) affects tomorrow's asset returns.

When performing this kind of analysis, this is the train of thought that leads to a solution:

1. The goal is to align a particular factor's results to future returns. 
2. Window length gets a range of dates starting from the current date in the pipeline minus `window_length` days. Therefore, for a `window length` of 2, for instance, the inputs in the `compute()` function in a Factor should start from the `today` argument minus 2 days.

In that case, when we want to compare today's sentiment and tomorrow's asset returns, we want to get news sentiment from `today - 2` and `today`'s asset returns.

In [ ]:
# Do not change anything in this code cell

from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.filters import StaticAssets
from quantopian.pipeline.factors import Returns, Latest

from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

assets = StaticAssets(symbols(['FB', 'TSLA', 'NFLX', 'AMZN', 'GOOG']))


start_date = datetime.strptime('01/01/2018', '%m/%d/%Y') + pd.tseries.offsets.BDay(2)
end_date = datetime.strptime('01/01/2019', '%m/%d/%Y')


First of all, get an overview of the available BoundColumns of the `sentiment` dataset.

In [ ]:
sentiment.columns

Build a custom factor to get sentiment signals with 2 days delay. 

In [ ]:
# Todo: Create a Sentiment factor.
class Sentiment(CustomFactor):
    ...

Build a pipeline

In [ ]:
# Todo: Build a pipeline with three columns.
return_d2 = ...
sentiment_d2 = ...

pipe = Pipeline(
    columns={
        'sentiment_today': ...,
        'sentiment_d2': ...,
        'return_d2': ...
    },
    screen=...
)

mdf = ...

## 1. Confirm that the Pipeline is correct

To confirm that our pipeline is correct, compare `sentiment_d2` with `sentiment_today`. The former should be delayed by 1 day, which means if 2018-01-03's NFLX's `sentiment_today` is 2.0, then 2018-01-04's NFLX's `sentiment_d2` should also be 2.0.

In [ ]:
mdf.head(15)

Since we have confirmed the values, delete `sentiment_today` column, we do not need this in our visualization.

In [ ]:
# Todo: Drop the `sentiment_today` column.
final_mdf = ...
final_mdf.head(5)

## 2. Plotting a Simple Correlation

[regplot](https://seaborn.pydata.org/generated/seaborn.regplot.html) can be used to quickly visualize correlation by using a regression plot.

In [ ]:
# Todo: Create a regplot.


There are 3 problems in that plot:

1. It is hard to see the density of the areas of data points. This is because we have a discrete data for `sentiment_d2`. One solution is to jitter the data points horizontally. Find out how to do this from the [regplot's documentation](https://seaborn.pydata.org/generated/seaborn.regplot.html).
2. The ticker spacing is not ideal for discrete `sentiment_d2` values. We will adjust the spacing using `xaxis.set_ticks()` function, passing in an array of numbers between -4 and 8 using [`np.arange`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html) function.

In [ ]:
# Todo: Add horizontal jittering and set the ticks on xaxis
#       so it shows all the sentiment scores.
import numpy as np

ax = ...
ax.xaxis.set_ticks(np.arange(...));

Zooming-in can be done with [`matplotlib.axes.Axes.set_ylim()`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.set_ylim.html) function

In [ ]:
# Todo: Zoom in with `set_ylim()` function
ax = ...
ax.xaxis.set_ticks(np.arange(...))
ax.set_ylim(...);

As it turned out, we do not see a conceivable pattern here. Let's group by sector and plot the regression for each sector to see if the trends are more significant when we compare them on per sector-basis.

## 3. Building a Multivariate Plot

In this section, you will create a multivariate plot, that is, a plot with three variables. The third variable here is the sectors. We will create a similar plot to the previous one, but use a different color for each sector.

### 3.1. Re-run the Pipeline + Sector Column

To do this, we need to re-run the pipeline, this time with the sector column included.

In [ ]:
# Do not change anything in this code cell

from quantopian.pipeline.classifiers.morningstar import Sector

In [ ]:
# Todo: Recreate the pipeline. Create and include Sector factor.

sector = Sector(mask=...)

sector_pipe = Pipeline(
    columns={
        'sentiment_d2': ...,
        'return_d2': ...,
        'sector': ...
    },
    screen=...
)

sector_mdf = ...

In [ ]:
# Todo: Create a regplot.
ax = ...
ax.xaxis.set_ticks(np.arange(...));

In [ ]:
sector_mdf.head(5)

### 3.2. Choose a Color Palette

Before visualizing multivariate plots, we first choose a color palette to use. Some of the available palettes are documented [here](https://seaborn.pydata.org/tutorial/color_palettes.html). Since we are dealing with discrete data, *paired* palette seems like a good idea.

First, get the list of sectors by getting unique values of `sector_mdf['sector']`, then use it as the number of colors.

In [ ]:
sectors = sector_mdf['sector'].unique()
pal = sns.color_palette('Paired', len(sectors))
sns.palplot(pal)

### 3.3. Visualize with lmplot

And finally, we can use `lmplot()` function to visualize all the fields in our DataFrame.

In [ ]:
# Todo: Create an lmplot.


## 3.4. Visualize with multiple regplots

For practice, draw the same plot with multiple regplots. Here are the tasks involved:

1. Prepare a legend elements list.
2. Loop over the list of sector codes. In each loop:
3. Get data for that sector using pandas selection by values.
4. Create a regplot with the data and color associated with the sector. Attach this plot to the Axes object passed through the arguments.
5. Add a Line2D instance to the list of legend elements.
6. (Outside of the loop) Attach a legend that contains the legend elements to the Axes object.

In [ ]:
# Todo: Complete the `draw_plot` function below.

def draw_plot(x, y, hue, data, ax, sectors, pal):
    # 1. Todo: Prepare a legend elements list.
    legend_elements = ...
    
    # 2. Todo: Loop over the list of sector codes.
    for i in ...:
        
        # 3. Todo: Get data for that sector using pandas selection by values.
        df = ...
        
        # 4. Todo: Create a regplot with the data and color associated 
        #          with the sector. Attach this plot to the Axes object
        #          passed through the arguments.
        color = ...
        facet = ...
        
        # 5. Todo: Add a Line2D instance to the list of legend elements.
        legend_elements.append(...)
    
    # 6. Todo: Attach a legend that contains the legend elements
    #          to the Axes object.
    ax.legend(...)
        

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1)
draw_plot('sentiment_d2', 'return_d2', 'sector', sector_mdf, ax, sectors, pal)